In [1]:
# Donwload and unzip the data

# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
# !unzip data.zip

### Importing libraires

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import models

from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### Making variables for the paths

In [4]:
train_path = '/home/jovyan/workspace/clothing-dataset-small/data/train'
test_path = '/home/jovyan/workspace/clothing-dataset-small/data/test'

### CNN Model

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [8]:
optimizer = optimizers.SGD(lr=0.002, momentum=0.8)

In [9]:
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

### Question 1

Since we have a binary classification problem, what is the best loss function for us?

* `mean squared error`
* `binary crossentropy`
* `categorical crossentropy`
* `cosine similarity`

> **Note:** since we specify an activation for the output layer, we don't need to set `from_logits=True`

- Answer question 1 =>  **binary crossentropy**

### Question 2

What's the number of parameters in the convolutional layer of our model? You can use the `summary` method for that. 

* 1 
* 65
* 896
* 11214912

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 175232)            0         
                                                                 
 dense_2 (Dense)             (None, 64)                11214912  
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
__________________________________________

- Answer question 2 =>  **896**

### Generators and Training

#### Train data

In [11]:
train_gen = ImageDataGenerator(rescale=1./255)

train_generator = train_gen.flow_from_directory(train_path,
                                                target_size=(150, 150),
                                                batch_size=20, 
                                                shuffle=True,
                                                class_mode='binary'
                                                )

Found 3677 images belonging to 2 classes.


#### Test data

In [12]:
test_gen = ImageDataGenerator(rescale=1./255)

test_generator = test_gen.flow_from_directory(test_path,
                                              target_size=(150, 150),
                                              batch_size=20,
                                              shuffle=True,
                                              class_mode='binary'
                                             )

Found 918 images belonging to 2 classes.


In [13]:
#@ INSPECTING:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


In [14]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10


2023-11-13 16:52:33.915099: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-11-13 16:52:34.538908: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-13 16:52:34.539686: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-13 16:52:34.539724: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-11-13 16:52:34.540589: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-13 16:52:34.540669: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


184/184 [==============================] - 13s 61ms/step - loss: 0.6726 - accuracy: 0.5668 - val_loss: 0.7040 - val_accuracy: 0.5370
Epoch 2/10
184/184 [==============================] - 8s 44ms/step - loss: 0.6212 - accuracy: 0.6478 - val_loss: 0.6414 - val_accuracy: 0.5621
Epoch 3/10
184/184 [==============================] - 8s 45ms/step - loss: 0.5774 - accuracy: 0.7006 - val_loss: 0.5635 - val_accuracy: 0.7288
Epoch 4/10
184/184 [==============================] - 8s 45ms/step - loss: 0.5428 - accuracy: 0.7278 - val_loss: 0.5562 - val_accuracy: 0.7222
Epoch 5/10
184/184 [==============================] - 8s 43ms/step - loss: 0.5081 - accuracy: 0.7582 - val_loss: 0.5232 - val_accuracy: 0.7440
Epoch 6/10
184/184 [==============================] - 8s 44ms/step - loss: 0.4891 - accuracy: 0.7737 - val_loss: 0.5279 - val_accuracy: 0.7582
Epoch 7/10
184/184 [==============================] - 8s 44ms/step - loss: 0.4628 - accuracy: 0.7884 - val_loss: 0.5229 - val_accuracy: 0.7495
Epoch 8/1

### Question 3

What is the median of training accuracy for all the epochs for this model?

* 0.20
* 0.40
* 0.60
* 0.80


In [15]:
import numpy as np

In [16]:
np.median(history.history['accuracy']).round(2)

0.77

- Answer question 3 =>  **0.77**

### Question 4

What is the standard deviation of training loss for all the epochs for this model?

- 0.031
- 0.061
- 0.091
- 0.131

In [19]:
np.std(history.history['loss']).round(3)

0.09

- Answer question 4 =>  **0.09**

### Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=50,`
* `width_shift_range=0.1,`
* `height_shift_range=0.1,`
* `zoom_range=0.1,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

In [20]:
train_gen = ImageDataGenerator(rescale=1./255,
                               rotation_range=50,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest')

train_generator = train_gen.flow_from_directory(train_path,
                                                target_size=(150, 150),
                                                batch_size=20, 
                                                shuffle=True,
                                                class_mode='binary'
                                                )

Found 3677 images belonging to 2 classes.


In [21]:
history_da = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
184/184 [==============================] - 23s 124ms/step - loss: 0.5001 - accuracy: 0.7604 - val_loss: 0.5023 - val_accuracy: 0.7647
Epoch 2/10
184/184 [==============================] - 23s 124ms/step - loss: 0.4900 - accuracy: 0.7656 - val_loss: 0.4848 - val_accuracy: 0.7854
Epoch 3/10
184/184 [==============================] - 23s 124ms/step - loss: 0.4797 - accuracy: 0.7745 - val_loss: 0.4895 - val_accuracy: 0.7800
Epoch 4/10
184/184 [==============================] - 23s 125ms/step - loss: 0.4865 - accuracy: 0.7699 - val_loss: 0.4760 - val_accuracy: 0.7887
Epoch 5/10
184/184 [==============================] - 23s 123ms/step - loss: 0.4759 - accuracy: 0.7759 - val_loss: 0.5072 - val_accuracy: 0.7691
Epoch 6/10
184/184 [==============================] - 23s 124ms/step - loss: 0.4747 - accuracy: 0.7800 - val_loss: 0.5164 - val_accuracy: 0.7723
Epoch 7/10
184/184 [==============================] - 23s 125ms/step - loss: 0.4609 - accuracy: 0.7922 - val_loss: 0.4739 - val_ac

### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
> **Note:** make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.18
* 0.48
* 0.78
* 0.108

In [22]:
np.mean(history_da.history['val_loss']).round(3)

0.486

- Answer question 5 =>  **0.486**

### Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

* 0.38
* 0.58
* 0.78
* 0.98

In [23]:
np.mean(history_da.history['val_accuracy'][5:10]).round(2)


0.78

- Answer question 6 =>  **0.78**